# Готовим данные для ARTM

В этом блокноте готовим данные, взятые из контакта для ARTM. Библиотека поддерживает [конкретные виды входа.](http://docs.bigartm.org/en/stable/tutorials/datasets.html) Будем перегонять их в **vowpal wabbit** формат, как наиболее хайповый. В нашей статье всё хайповое и трендовое. Также будем чистить данные от всякой мерзости и лемматизировать. 

In [1]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами

# Пакет для красивых циклов. При желании его можно отключить. 
# Тогда из всех циклов придётся  удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

import re      # пакет для работы с регулярными выражениями 
import pickle  # пакет для подгрузки данных специфического для питона формата

# работа со временем
from datetime import datetime

# Токенизатор
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Лемматизатор (приводит слова к нормальной форме)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Список стоп-слов
from nltk.corpus import stopwords
stop_ru = stopwords.words('russian')
stop_en = stopwords.words('english')
stop = stop_ru + stop_en

# Для распараллеливания кода
from joblib import Parallel, delayed

# Для перемешивания
from random import shuffle

import os

In [2]:
# Путь к данным
path = '/Users/dmitrys/Yandex.Disk.localized/top_russian_music/comments/by_styles/several_styles'

# жанры 
music = ['classic', 'rap', 'shanson', 'jazz', 'dance', 'estrada', 'indi', 'metal', 'pank', 'popsa', 'rock']

In [3]:
def dataLoad_(music_name, path = path, data_type='group_comments'):
    """ 
        Подгружает что-нибудь эдакое
        music_name: string
            название жанра
        path: string
            путь до директории с данными
        data_type
            суффикс файла (group_comments - комментарии, users_wall, ...) 
    """
    # подгружаем данные 
    with open(path +'/' + music_name + '_' + data_type, 'rb') as f:
        df = pickle.load(f)
    return df

In [4]:
def dataLoadv2(music_name, path=path, data_type='group_comments'):
    """ 
        Подгружает что-нибудь эдакое
        music_name: string
            название жанра
        path: string
            путь до директории с данными
        data_type
            суффикс файла (group_comments - комментарии, users_wall, ...) 
    """
    # находим все файлы директории
    files = os.listdir(path='{}/{}/'.format(path, music_name))
    files = [f for f in files if data_type in f]
    
    data = {}
    # подгружаем данные 
    for file in files:
        group_name = file.split(data_type+'_')[-1]
        with open('{}/{}/{}'.format(path, music_name, file), 'rb') as f:
            df = pickle.load(f)
        data[group_name] = df
    
    return data

In [5]:
def dataLoad(music_name, path=path, data_type='group_comments'):
    try:
        with open(path + '/'+ music_name + '_' + data_type, 'rb') as f:
            df = pickle.load(f)
        return df
    except:
        # находим все файлы директории
        files = os.listdir(path='{}/{}/'.format(path, music_name))
        files = [f for f in files if data_type in f]

        data = {}
        # подгружаем данные 
        for file in files:
            group_name = file.split(data_type+'_')[-1]
            with open('{}/{}/{}'.format(path, music_name, file), 'rb') as f:
                df = pickle.load(f)
            data[group_name] = df
        return data

In [6]:
def htmlStrip(text):
    """ 
        Возвращает текст, очещенный от html тэгов
        text: string
            Текст поста 
    """
    
    return re.sub('<[^<]+?>', '', str(text)) 


def tokenLemma(text, stop = stop):
    """ 
        Возвращает список слов, очищенный от тэгов, пролемматизированный и без стоп слов
        text: string
            Текст поста
        
        parameters: list 
            stop: список из стоп-слов, example: ['политик', 'выбирать']
    """
    
    # понижение регистра, очистка от тэгов, токенизация
    out_1 = tokenizer.tokenize(htmlStrip(text.lower()))
    # лемматизация и очистка от стоп-слов, ещё выкидывает слова короче 3 символов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1 if tx not in stop and len(tx) > 2]
    return ' '.join(out_2)

## 1. Предобработка комментариев

В конечном итоге нам нужно, чтобы каждый комментарий перегонялся в формат **vowpal wabbit:**

```
'popsa_бузова_4887563_208109_208174 |@default_class id289877716 глеб публикация какой какой число день какой коммент ольга ответить это плохой комментатор один видеть олечка ответить коммент|@likes 0 |@stickers sticker_1 sticker_2  |@audio николай басков шарманка |@video клип басков шарманка |@link плейлист киркоров |@emoji 😊😊😊'
```

Модальности: 

* имя коммента `жанр исполнитель id группы_id поста_id коммента`
* текст коммента 
* число лайков
* стикеры в комменте 
* название аудиозаписи, вложенной в коммент 
* название и описание видеозаписи, вложенной в коммент 
* ссылка на плейлист или ещё куда и её описание
* эмодзи, которые были в комменте

Именно в таком виде мы будем подавать комменты в модель для обучения. Кроме такого формата будем делать прмоежуточный: обычный массив из словарей с комментами. Его можно будет быстро отфильтровать по какому-то признаку, превратить в табличку и тп. Именно его будем использовать для сентимент-модели и матерных списков Олечки. 

In [20]:
def firstCommentPrepare(comment, style, performer):
    """
        Первичная предобработка комментария в словарик
    """
    
    # интересные вложения
    stickers = [ ]  # стикеры 
    audio = [ ]     # музыка 
    video = [ ]     # видяшки
    # photo_cnt = 0   # счётчик для фоточек с красоточками
    # doc_cnt = 0     # чаще всего это фотки либо гифки
    link = [ ]      # ссылки на плейлисты или ещё куда
        
    if isinstance(comment, str):
        print(comment)
        return None

    attach = comment.get('attachments')
    if attach is not None:
        for at in attach:
            tp = at['type']
            if tp == 'sticker':
                stickers.append(tp + '_' + str(at[tp]['sticker_id']))
            elif tp == 'audio':
                audio.append(at[tp]['artist'] + ' ' + at[tp]['title'])
            elif tp == 'video':
                # из интересных вложений есть ещё shows, comments, duration
                video.append(at[tp]['title'] + ' ' + at[tp]['description'])
            # elif tp == 'photo':
            #     photo_cnt += 1
            # elif tp == 'doc':
            #     doc_cnt += 1
            elif tp == 'link':
                link.append(at[tp]['title'] + ' ' + at[tp]['description']) 
                
    # Выбрасываем из записи все символы, кроме эмодзи и редких символов вроде иероглифов         
    emoji = re.sub('[a-яa-zA-ZА-Я -.,0-9<>:=;/_—@$^?–ё`%#*&!№…\|/\"\]\[\"\'\n\t”“’‘]','',comment['text'])
                    
    comment_infa = {     
        'music_style' : style,
        'performer' : performer,
        'comment_id' :  str(comment['group_id']) + '_' + str(comment['post_id'])  + '_' + str(comment['id']),
        'author' : str(comment['from_id']),
        'likes' : str(comment['likes']),
        'date' : datetime.utcfromtimestamp(comment['date']).strftime('%Y-%m-%d %H:%M:%S'),
        'text' : tokenLemma(comment['text']),
        'dirty_text' : comment['text'],
        'emoji' : emoji if len(emoji) > 1 else None,
        'stickers' : ' '.join(stickers) if len(stickers) > 0 else None, 
        'audio' : tokenLemma(' '.join(audio)) if len(audio) > 0 else None,
        'video' : tokenLemma(' '.join(video)) if len(video) > 0 else None,
        'link' : tokenLemma(' '.join(link)) if len(link) > 0 else None
        }
    return comment_infa


def secondCommentPrepare(comment, modality=True):
    """
    Вторичная предобработка комментариев из словаря в vw
        comment : string 
            словарик с комментарием
        modality : bool
            нужны ли модальности по видео, аудио, ссылкам и тп
    """
    # инфу про автора и дату я нигде не использую 
    comment_name = comment['music_style'] + '_' + comment['performer'] + '_' + comment['comment_id']
    likes = ' |likes {} '.format(comment['likes'])
    text = ' |default_class {}'.format(comment['text'])
    
    # базовая инфа для модели 
    comment_body = comment_name + text  + likes
    
    # дополнительные модальности 
    if modality:
        if comment['stickers']:
            comment_body = comment_body + ' |stickers ' + comment['stickers']
        if comment['audio']:
            comment_body = comment_body + ' |audio ' + comment['audio']
        if comment['video']:
            comment_body = comment_body + ' |video ' + comment['video']
        if comment['link']:
            comment_body = comment_body + ' |link ' + comment['link']
        if comment['emoji']:
            comment_body = comment_body + ' |emoji ' + comment['emoji']
        
            
    return comment_body

In [ ]:
music = ['classic', 'rap', 'shanson', 'jazz', 'dance', 'estrada', 'indi', 'metal', 'pank', 'popsa', 'rock']

In [39]:
df = dataLoad_('estrada')

In [40]:
type(df)

dict

In [28]:
test = firstCommentPrepare(df['jazz_music'][16],'jazz','jazz_music')
test

{'music_style': 'jazz',
 'performer': 'jazz_music',
 'comment_id': '81892801_3079_3150',
 'author': '-164643020',
 'likes': '1',
 'date': '2018-10-16 15:08:07',
 'text': '',
 'dirty_text': '',
 'emoji': None,
 'stickers': None,
 'audio': 'first station groove original mix original mix',
 'video': None,
 'link': None}

In [29]:
secondCommentPrepare(test)

'jazz_jazz_music_81892801_3079_3150 |default_class  |likes 1 '

In [30]:
secondCommentPrepare(test, modality=True)

'jazz_jazz_music_81892801_3079_3150 |default_class  |likes 1  |audio first station groove original mix original mix'

In [ ]:
comments = [ ]

In [43]:
music = ['indi', 'metal', 'pank', 'popsa', 'rock']

for name in music:
    df = dataLoad(name)
    
    for key in tqdm_notebook(df):
        print('Предобрабатывают',name)
        # Внимание, костыль! 
        #def prCom(w):
        #    return firstCommentPrepare(w, name, key)
        
        n_jobs = -1 # параллелим на все ядра 
        result = Parallel(n_jobs=n_jobs)(delayed(firstCommentPrepare)(text, name, key) for text in tqdm_notebook(df[key]))
        comments.extend(result)
        print('Текущее число комментариев:', len(comments))
        

Предобрабатывают indi


Текущее число комментариев: 3349352
Предобрабатывают indi


Текущее число комментариев: 3370684
Предобрабатывают indi


Текущее число комментариев: 3374082
Предобрабатывают indi


Текущее число комментариев: 3374976
Предобрабатывают indi


...

Текущее число комментариев: 9248877
Предобрабатывают rock


Текущее число комментариев: 9259574
Предобрабатывают rock


Текущее число комментариев: 9269030
Предобрабатывают rock


Текущее число комментариев: 9294180
Предобрабатывают rock


Текущее число комментариев: 9297753


In [ ]:
print(len(comments))

In [15]:
#shuffle(comments) # перемешаем комменты
comments[0]

{'music_style': 'estrada',
 'performer': 'лепс',
 'comment_id': '52228413_282387_282388',
 'author': '134973383',
 'likes': '21',
 'date': '2018-12-31 21:40:35',
 'text': 'шикарный подарок песня замечательный спасибоооо новый год друг',
 'dirty_text': 'Шикарный подарок , песня замечательная !!!! Спасибоооо !! С Новым годом всех друзья!!!!',
 'emoji': None,
 'stickers': None,
 'audio': None,
 'video': None,
 'link': None}

In [9]:
path = '/Users/dmitrys/Yandex.Disk.localized/top_russian_music/comments/by_styles/'

In [11]:
with open(path+'/vk_comments_full_1', "rb") as fin:
    comments = pickle.load(fin)

In [16]:
with open(path+'/vk_comments_full_2', "rb") as fin:
    comments.extend(pickle.load(fin))

In [17]:
len(comments)

9297743

In [12]:
comments = [com for com in comments if com!=None]

In [18]:
# отфильтруем слишком короткие комменты (меньше 10 символов) 
# тут же надо будет сделать сурьёзные махинации с отбором комментов по длине и сэмплом по жанрам 
comments = [com for com in comments if len(com['text']) > 10]
len(comments)

6655928

In [19]:
with open('{}/vk_comments_big.json'.format(path), 'w') as outfile:  
    json.dump(comments, outfile)

In [21]:
secondCommentPrepare(comments[0])

'estrada_лепс_52228413_282387_282388 |default_class шикарный подарок песня замечательный спасибоооо новый год друг |likes 21 '

In [22]:
n_jobs = -1 # параллелим на все ядра 
artm_result = Parallel(n_jobs=n_jobs)(delayed(secondCommentPrepare)(
    text) for text in tqdm_notebook(comments))

In [23]:
artm_result[0]

'estrada_лепс_52228413_282387_282388 |default_class шикарный подарок песня замечательный спасибоооо новый год друг |likes 21 '

In [24]:
# сохраняем полученное добро 
with open('{}/vk_comments_big_artm'.format(path), 'a') as out:
    for item in tqdm_notebook(artm_result):
        out.write("{}\n".format(item))

In [8]:
with open('../data_artm/classic_rap_shanson_jazz_comments_v1', 'r') as f:
    comments = f.read()

In [ ]:
comments[:100]

## 2. Предобработка стенок юзеров

In [ ]:
# задел на будущее

## 3. Предобраьотка постов из групп

In [ ]:
# задел на будущее